# EC2(continued) : run Jupyter Notebook

## on AWS EC2:
- after changing the instance status to "running", copy the public DNS of it<br/>

## With Powershell :

- Move to the directory where the key pair is located<br/>
`$ ssh -i "<your-key-pair>" ubuntu@<your-public-DNS>`<br/>
- Install Anaconda<br/>
`$ wget https://repo.continuum.io/archive/Anaconda3-4.4.0-Linux-x86_64.sh`<br/>
`$ bash Anaconda3-4.4.0-Linux-x86_64.sh`<br/>
- Check whether Path is correctly input with Vim<br/>
`$ vi ~/.bashrc`<br/>
`export PATH="/home/ec2-user/anaconda3/bin:$PATH"`<br/>
- Check whether Python runs properly through Anaconda<br/>
`$ python`<br/>
- Run ipython and enter password<br/>
`$ ipython`<br/>
- Copy SHA string <br/>
- Edit jupyter notebook config <br/>
`$ jupyter notebook --generate-config`<br/>
`$ mkdir certs && cd certs`<br/>
`$ sudo openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mycert.pem -out mycert.pem`<br/>
`$ vi .jupyter/jupyter_notebook_config.py`<br/>

```
c = get_config()

# Kernel config
c.IPKernelApp.pylab = 'inline'

# Notebook config
c.NotebookApp.certfile = u'/home/ubuntu/certs/mycert.pem' #location of your certificate file
c.NotebookApp.ip = '*'
c.NotebookApp.open_browser = False  
c.NotebookApp.password = u'<your-SHA-string>'  

c.NotebookApp.port = 8888
```

`$ mkdir dev/pynbs && cd dev/pynbs`<br/>
`$ jupyter notebook`<br/>

## On Browser:
- Open jupyter notebook<br/>
`https://<your-public-DNS>:8888/`<br/>

## Now you can connect jupyter notebook:
- Let the instance start running and copy your public DNS<br/>
- Move to the directory where the key pair is located<br/>
- And<br/>
`$ ssh -i "<your-key-pair>" ubuntu@<your-public-DNS>`<br/>
`$ ls` for check<br/>
`$ cd ipynbs`<br/>
`$ jupyter notebook`<br/>
`https://<your-public-DNS>` <br/>
- To shut down: simply stop the instance on browser<br/>

# Flask

## Web Frameworks Built with Python
- Full-stack<br/>
Django<br/>
Pyramid<br/>
Web2py<br/>
- Microframework<br/>
**Flask**<br/>
Bottle<br/>
- Async <br/>
Tornado<br/>
Sanic<br/>

## Install Flask
`$ pip install flask`

## Hello World with Flask
- server.py

```
from flask import Flask


app = Flask(__name__)

@app.route('/')
def index():
	return 'hello world!'
    
if __name__ == '__main__':
	app.run(host='0.0.0.0', port=8080, debug=True)
```

`$ python server.py`

## Flask-route

```
from flask import Flask


app = Flask(__name__)

@app.route('/')
def index():
	return 'hello'

@app.route('/about')
def about():
	return 'about'

if __name__ == '__main__':
	app.run(host='0.0.0.0', port=8080, debug=True)
```

## Flask-render

- Inspect templates<br/>
```
/
  server.py
  /templates
    index.html
    about.html
```

- on Editor

```
from flask import Flask, render_template


app = Flask(__name__)

@app.route('/')
def index(name=None):
	return render_template('index.html', name=name)

@app.route('/about')
def about(name=None):
	return render_template('about.html', name=name)

if __name__ == '__main__':
	app.run(host='0.0.0.0', port=8080, debug=True)
```

## Flask with BeautifulSoup
```
from flask import Flask
from bs import BeautifulSoup
from time import ctime
import requests

app = Flask(__name__)

@app.route('/')
def index():
    naver_uri = "https://www.naver.com"
    executed_time = ctime()
    naver_html = requests.get(naver_uri).text
    
    soup = BeautifulSoup(naver_html, 'html.parser')
    kw_ul = soup.find('ul', attrs={'class':'ah_l'})
    keywords = [li.text for li in kw_ul.find_all('span', attrs={'class': 'ah_k'})]

    return render_template('index.html', time=executed_time, keywords=keywords)
    
if __name__=='__main__':
    app.run(host='0.0.0.0', port=80, debug=False)
```

## Route with querystring, path
```
from flask import Flask, render_template, request


app = Flask(__name__)

@app.route('/user/<string:name>')
def user(name=None):
    return render_template('user.html', msg=name)
    
@app.route('/users')
def users():
    querystring = request.args
    return render_template('users.html', rows=querystring)
if __name__=='__main__':
    app.run(host='0.0.0.0', port=80, debug=False)
```

## SQL with SQLite
```
from flask import Flask, render_template, request
import sqlite3 as lite


app = Flask(__name__)

@app.route('/movies', methods=["GET", "POST"])
def movies():
    if request.method == "GET":
        try:
            conn = lite.connect('./data/data.db')
            conn.row_factory = lite.Row
            
            cur = conn.cursor()
            query = "SELECT * FROM Movies;"
            cur.execute(query)

            rows = cur.fetchall()
            conn.close()

            return render_template('movies.html', rows=rows)
        except:
            pass

    elif request.method == "POST":
        try:
            input_name = request.form["movie-name"]
            input_year = request.form["movie-year"]
            input_studio = request.form["movie-studio"]

            with lite.connect('./data/data.db') as conn:
                cur = conn.cursor()
                query = """
                    INSERT INTO Movies(name, year, studio)
                    VALUES(?,?,?)
                """
                cur.execute(query,
                        (input_name, 
                         input_year, 
                         input_studio,
                         )
                        )
                conn.commit()
                msg = "Success"
        except:
            conn.rollback()
            msg = "Failed"
        finally:
            return render_template('movies.html', msg=msg)
            
if __name__=='__main__':
    app.run(host='0.0.0.0', port=80, debug=False)
```

## Simple API Server with Flask, mLab
```
from flask import Flask, jsonify
import requests
from pymongo import MongoClient

app = Flask(__name__)

mongo_uri = "mongodb://<mLabusername>:<mLabpassword>@ds145299.mlab.com:45299/mydbinstance"

@app.route('/api/v1/item')
def api():
    client = MongoClient(mongo_uri)
    db = client.mydbinstance
    items = db.bigbang
    try:
        query = {}
        projection= {
                "_id":0,
                "title":1,
                "item":1,
                }
        result = list(items.find(query, projection))
    except:
        result = "failed"
    finally:
        return jsonify({"items":result})
        
if __name__=='__main__':
    app.run(host='0.0.0.0', port=80, debug=False)
```

# EC2 with Flask
- Create repo on Github and clone it<br/>
`$ git clone <copied-repo-address>`<br/>

```
if __name__=='__main__':
    app.run(host='0.0.0.0', port=80, debug=False)
```

- Run server.py<br/>
`$ python server.py`